# HỒI QUY SOFTMAX (MULTINOMIAL LOGISTIC REGRESSION)

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from scipy import sparse
from sklearn.preprocessing import label_binarize

## Hàm softmax 

In [12]:
def convert_labels(y, C):
    Y = sparse.coo_matrix((np.ones_like(y), (y, np.arange(len(y)))), shape=(C, len(y))).toarray()
    return Y

def softmax_stable(Z):
    e_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return e_Z / e_Z.sum(axis=0)

def softmax(Z):
    e_Z = np.exp(Z)
    return e_Z / e_Z.sum(axis=0)

def softmax_regression(X, y, W_init, eta=0.05, tol=1e-4, max_count=10000):
    W = [W_init]
    C = W_init.shape[1]
    Y = convert_labels(y, C)
    N = X.shape[1]
    d = X.shape[0]

    count = 0
    check_w_after = 1500
    while count < max_count:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(np.dot(W[-1].T, xi))
            W_new = W[-1] + eta * xi.dot((yi - ai).T)
            count += 1
            if count % check_w_after == 0:
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W
            W.append(W_new)
    return W

#Thêm hiệu chỉnh L2 
# def softmax_regression(X, y, W_init, eta=0.05, tol=1e-4, max_count=10000, lambda_reg=0.01):
#     W = [W_init]
#     C = W_init.shape[1]
#     Y = convert_labels(y, C)
#     N = X.shape[1]
#     d = X.shape[0]

#     count = 0
#     check_w_after = 1500
#     while count < max_count:
#         mix_id = np.random.permutation(N)
#         for i in mix_id:
#             xi = X[:, i].reshape(d, 1)
#             yi = Y[:, i].reshape(C, 1)
#             ai = softmax(np.dot(W[-1].T, xi))

#             # Cập nhật trọng số với regularization
#             grad = xi.dot((yi - ai).T) - lambda_reg * W[-1]  # L2 regularization
#             W_new = W[-1] + eta * grad

#             count += 1
#             if count % check_w_after == 0:
#                 if np.linalg.norm(W_new - W[-check_w_after]) < tol:
#                     return W
#             W.append(W_new)
#     return W


In [13]:
# dự đoán và đánh giá các chỉ số 
def pred(W, X):
    A = softmax_stable(W.T.dot(X))
    return np.argmax(A, axis=0)

def evaluate_model(y_true, y_pred):
    print(classification_report(y_true, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred), average='macro', multi_class='ovr')
    print(f"AUC (macro average): {auc:.4f}")

## DỮ LIỆU GỐC 

In [14]:
# Load dữ liệu
df = pd.read_csv("../../data/data_processed/data_processed.csv")

# Tách đặc trưng và nhãn
X = df.drop(columns=["NSP"]).values
y = df["NSP"].values - 1  # Chuyển về 0,1,2

# Chuẩn hóa
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Thêm bias
X_scaled = np.hstack([np.ones((X_scaled.shape[0], 1)), X_scaled])
C = len(np.unique(y))

In [15]:
# Hàm huấn luyện và đánh giá với tỉ lệ cho trước
from sklearn.preprocessing import StandardScaler

def train_and_evaluate(X, y, test_size):
    print(f"\n--- Tỉ lệ train:test = {int((1-test_size)*10)}:{int(test_size*10)} ---")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

    X_train_T = X_train.T
    X_test_T = X_test.T
    d = X_train_T.shape[0]

    W_init = np.random.randn(d, C)
    W = softmax_regression(X_train_T, y_train, W_init)[-1]

    y_pred = pred(W, X_test_T)
    evaluate_model(y_test, y_pred)


# Chạy với các tỉ lệ
for test_size in [0.2, 0.3, 0.4]:
    train_and_evaluate(X, y, test_size)



--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.92      0.95      0.94       332
         1.0       0.65      0.61      0.63        59
         2.0       0.85      0.66      0.74        35

    accuracy                           0.88       426
   macro avg       0.81      0.74      0.77       426
weighted avg       0.88      0.88      0.88       426

Confusion Matrix:
[[317  12   3]
 [ 22  36   1]
 [  5   7  23]]
AUC (macro average): 0.8121

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94       497
         1.0       0.59      0.61      0.60        88
         2.0       0.87      0.64      0.74        53

    accuracy                           0.88       638
   macro avg       0.80      0.74      0.76       638
weighted avg       0.88      0.88      0.88       638

Confusion Matrix:
[[474  20   3]
 [ 32  54   2]
 [  2  17  34]]
AUC (macro average

Tỷ lệ train:test = 8:2
Accuracy: 0.87
F1-score (macro): 0.74
AUC (macro): 0.8165

Tỷ lệ train:test = 7:3
Accuracy: 0.86
F1-score (macro): 0.74
AUC (macro): 0.8325

Tỷ lệ train:test = 6:4
Accuracy: 0.89 → cao nhất trong 3 tỷ lệ.
F1-score (macro): 0.78 → cũng cao nhất.
AUC (macro): 0.8344

Nhận xét:
Tỷ lệ train:test = 6:4 cho kết quả tốt nhất tổng thể (Accuracy, F1, AUC đều cao).


Lớp 1 (suspect) thường có precision và recall thấp do đặc điểm đặc trưng của nó nằm giữa hai lớp còn lại (normal và pathologic), khiến mô hình dễ nhầm lẫn. Đây là lớp trung gian về mặt y học, nên các đặc trưng không phân tách rõ ràng, dẫn đến việc mô hình thường dự đoán sai sang lớp 0 hoặc lớp 2. Đồng thời, softmax regression là mô hình tuyến tính nên càng gặp khó khăn khi ranh giới giữa các lớp không rõ ràng hoặc có sự chồng lấn trong không gian đặc trưng.

## DỮ LIỆU GIẢM CHIỀU 

### GIẢM TRƯỚC CHIA SAU 

### PCA 

In [16]:
pca_df = pd.read_csv("../../data/dimension_reduction/pca/pca_all.csv")
pca_X = pca_df.drop(columns=["NSP"]).values
pca_y = pca_df["NSP"].values - 1
scaler = StandardScaler()
pca_X_scaled = scaler.fit_transform(pca_X)

print("--- Đánh giá với PCA ---")
for test_size in [0.2, 0.3, 0.4]:
    train_and_evaluate(pca_X_scaled, pca_y, test_size)

--- Đánh giá với PCA ---

--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95       332
         1.0       0.68      0.58      0.62        59
         2.0       0.81      0.74      0.78        35

    accuracy                           0.89       426
   macro avg       0.81      0.76      0.78       426
weighted avg       0.89      0.89      0.89       426

Confusion Matrix:
[[320  10   2]
 [ 21  34   4]
 [  3   6  26]]
AUC (macro average): 0.8281

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95       497
         1.0       0.58      0.69      0.63        88
         2.0       0.91      0.58      0.71        53

    accuracy                           0.88       638
   macro avg       0.81      0.74      0.76       638
weighted avg       0.89      0.88      0.88       638

Confusion Matrix:
[[471  25   1]
 [ 25  61   2]
 [  3  19

### LDA 

In [17]:
lda_df = pd.read_csv("../../data/dimension_reduction/lda/lda_all.csv")

lda_X = lda_df.drop(columns=["NSP"]).values
lda_y = lda_df["NSP"].values - 1
lda_X_scaled = scaler.fit_transform(lda_X)

print("\n--- Đánh giá với LDA ---")
for test_size in [0.2, 0.3, 0.4]:
  train_and_evaluate(lda_X_scaled, lda_y, test_size)


--- Đánh giá với LDA ---

--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95       332
         1.0       0.60      0.59      0.60        59
         2.0       0.82      0.51      0.63        35

    accuracy                           0.88       426
   macro avg       0.78      0.69      0.73       426
weighted avg       0.87      0.88      0.87       426

Confusion Matrix:
[[321   9   2]
 [ 22  35   2]
 [  3  14  18]]
AUC (macro average): 0.7893

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95       497
         1.0       0.65      0.60      0.63        88
         2.0       0.85      0.66      0.74        53

    accuracy                           0.89       638
   macro avg       0.81      0.74      0.77       638
weighted avg       0.89      0.89      0.89       638

Confusion Matrix:
[[480  15   2]
 [ 31  53   4]
 [  5  1

### So sánh và đánh giá kết quả
1. Dữ liệu gốc
- Accuracy: Tăng dần từ 0.87 → 0.89 khi tăng tỷ lệ dữ liệu huấn luyện.

- Macro F1-score: Giao động quanh 0.74 → 0.78, tức là mô hình cân bằng khá tốt giữa các lớp.

- AUC (macro): 0.8165 → 0.8344 — ổn định và tương đối cao.

Ưu điểm:

- Hiệu suất tổng thể khá tốt, đặc biệt với lớp 0.

- Recall của lớp 1 (tập nhỏ) khá cao, đặc biệt khi test size lớn (0.81 ở tỷ lệ 7:3).

Nhược điểm:

- Precision và Recall cho lớp 2 chưa ổn định.

- Có dấu hiệu mô hình học tốt lớp chiếm số đông, nhưng lớp ít (1, 2) vẫn chưa thực sự tốt.

2. Dữ liệu PCA
- Accuracy: Giảm còn ~0.74–0.77, tức là mô hình yếu hơn rõ rệt.

- Macro F1-score: Chỉ còn 0.60–0.65, thấp hơn dữ liệu gốc.

- AUC (macro): Tụt xuống khoảng 0.78–0.81.

Nhận xét:

- PCA là phương pháp giảm chiều không sử dụng nhãn lớp, nên thông tin phân biệt lớp bị mất.

- Mô hình bị giảm hiệu năng rõ rệt, đặc biệt là với lớp 2 (precision và recall thấp).

3. Dữ liệu LDA
- Accuracy: Tăng nhẹ so với PCA (~0.76–0.77), nhưng vẫn thấp hơn dữ liệu gốc.

- Macro F1-score: Giao động từ 0.62–0.65, khá sát với PCA.

- AUC (macro): Đạt ~0.8088–0.8241, tốt hơn PCA, gần bằng dữ liệu gốc.

Ưu điểm:

- Recall lớp 1 và lớp 2 tăng rõ rệt, nhờ LDA tận dụng nhãn lớp khi giảm chiều.

- Mô hình học tốt hơn so với PCA.

Nhược điểm:

- Precision cho lớp 2 vẫn thấp → mô hình vẫn chưa học tốt lớp ít dữ liệu.

### Nhận xét chung mô hình softmax: 
Lớp 0 (chiếm đa số) luôn có precision và recall rất cao → mô hình dễ thiên lệch về lớp đa số.

Khi tăng dữ liệu huấn luyện (giảm test size), mô hình thường có Recall lớp nhỏ tốt hơn, cho thấy mô hình cần nhiều dữ liệu để học tốt lớp thiểu số.

Có dấu hiệu thiên lệch (bias) lớp, tức là không cân bằng trong phân loại.

## CHIA TRƯỚC GIẢM SAU 

In [18]:
def train_test_from_file(train_path, test_path, test_ratio_label):
    print(f"\n--- Tỉ lệ train:test = {test_ratio_label} ---")
    
    # Đọc dữ liệu
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    X_train = train_df.drop(columns=['Unnamed: 0', "NSP"], axis=1).values.T
    y_train = train_df['NSP'].values - 1
    X_test = test_df.drop(columns=['Unnamed: 0', "NSP"], axis=1).values.T
    y_test = test_df['NSP'].values - 1

    # Huấn luyện
    d = X_train.shape[0]
    W_init = np.random.randn(d, C)
    W = softmax_regression(X_train, y_train, W_init)[-1]

    # Dự đoán và đánh giá
    y_pred = pred(W, X_test)
    evaluate_model(y_test, y_pred)

In [19]:
pca_files = [
    ("../../data/dimension_reduction/pca/train_80.csv", "../../data/dimension_reduction/pca/test_20.csv", "8:2"),
    ("../../data/dimension_reduction/pca/train_70.csv", "../../data/dimension_reduction/pca/test_30.csv", "7:3"),
    ("../../data/dimension_reduction/pca/train_60.csv", "../../data/dimension_reduction/pca/test_40.csv", "6:4"),
]

for train_file, test_file, label in pca_files:
    train_test_from_file(train_file, test_file, label)



--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.97      0.77      0.86       332
         1.0       0.41      0.75      0.53        59
         2.0       0.36      0.54      0.43        35

    accuracy                           0.75       426
   macro avg       0.58      0.69      0.61       426
weighted avg       0.84      0.75      0.78       426

Confusion Matrix:
[[257  47  28]
 [  9  44   6]
 [  0  16  19]]
AUC (macro average): 0.7847

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.97      0.75      0.85       497
         1.0       0.47      0.78      0.59        88
         2.0       0.38      0.75      0.50        53

    accuracy                           0.76       638
   macro avg       0.61      0.76      0.65       638
weighted avg       0.85      0.76      0.78       638

Confusion Matrix:
[[373  66  58]
 [ 11  69   8]
 [  1  12  40]]
AUC (macro average

In [20]:
lda_files = [
    ("../../data/dimension_reduction/lda/train_80.csv", "../../data/dimension_reduction/lda/test_20.csv", "8:2"),
    ("../../data/dimension_reduction/lda/train_70.csv", "../../data/dimension_reduction/lda/test_30.csv", "7:3"),
    ("../../data/dimension_reduction/lda/train_60.csv", "../../data/dimension_reduction/lda/test_40.csv", "6:4")
]

for train_file, test_file, label in lda_files:
    train_test_from_file(train_file, test_file, label)



--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.97      0.78      0.87       332
         1.0       0.50      0.75      0.60        59
         2.0       0.40      0.80      0.53        35

    accuracy                           0.78       426
   macro avg       0.62      0.78      0.67       426
weighted avg       0.86      0.78      0.80       426

Confusion Matrix:
[[260  37  35]
 [  8  44   7]
 [  0   7  28]]
AUC (macro average): 0.8361

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.99      0.76      0.86       497
         1.0       0.44      0.88      0.59        88
         2.0       0.41      0.62      0.50        53

    accuracy                           0.77       638
   macro avg       0.61      0.75      0.65       638
weighted avg       0.87      0.77      0.79       638

Confusion Matrix:
[[380  77  40]
 [  4  77   7]
 [  0  20  33]]
AUC (macro average